In [1]:
import pandas as pd
df = pd.read_csv('iphone6-negative.csv')
df.head()

,TweetID,User_ID,Text,Sentiment
0,6.470000e+17,Rionagh,I've had an IPhone for like 3 years and I've n...,negative
1,6.470000e+17,your highness,I dont need the new iphone but I want it :(,negative
2,6.470000e+17,Little Liggins,fuck the iPhone 6s cus I'm not getting one :(,negative
3,6.470000e+17,NaN,Hopefully will be able to get my iPhone 6s tod...,negative
4,6.470000e+17,Kenny Tosh,@clydesdalebank I've just bought a new iPhone ...,negative


In [3]:
df['Sentiment'].value_counts()

negative    501
positive    501
Name: Sentiment, dtype: int64

In [4]:
df = df[['Text','Sentiment']]
df.head()

,Text,Sentiment
0,I've had an IPhone for like 3 years and I've n...,negative
1,I dont need the new iphone but I want it :(,negative
2,fuck the iPhone 6s cus I'm not getting one :(,negative
3,Hopefully will be able to get my iPhone 6s tod...,negative
4,@clydesdalebank I've just bought a new iPhone ...,negative


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(df['Text'])
x.shape

(1002, 3152)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
def train(x,y):
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=110)
    clf = LogisticRegression()
    clf.fit(x_train,y_train)
    return round(clf.score(x_test,y_test)*100,2)

In [8]:
train(x,df['Sentiment'])

89.05

## Stopwords removal

In [14]:
sw = open('sw.txt','r').read().split('\n') ## \n --> refer to new line
def sw_removal(text): ## text refer to whole sentence
    wanted_words = []
    for word in text.lower().split():
        if word not in sw:
            ## wanted word
            wanted_words.append(word)
    return ' '.join(wanted_words) ## combine list to form sentence

In [15]:
sw_removal("I've had an IPhone for like 3 years and I've")

'iphone like 3 years'

In [17]:
df['clean_text'] = df.Text.apply(sw_removal)
df.head()

,Text,Sentiment,clean_text
0,I've had an IPhone for like 3 years and I've n...,negative,iphone like 3 years never done :( hate ma life
1,I dont need the new iphone but I want it :(,negative,dont need new iphone want :(
2,fuck the iPhone 6s cus I'm not getting one :(,negative,fuck iphone 6s cus i'm getting one :(
3,Hopefully will be able to get my iPhone 6s tod...,negative,hopefully able get iphone 6s today :( plane ge...
4,@clydesdalebank I've just bought a new iPhone ...,negative,@clydesdalebank bought new iphone can't use ap...


In [18]:
x = vectorizer.fit_transform(df['clean_text'])
x.shape

(1002, 3090)

In [19]:
train(x,df.Sentiment)

88.56

## Denoise

In [22]:
import re
re.sub('[^a-zA-Z0-9 ]',' ',"I've had an IPhone for like 3 years and I've n...")

'I ve had an IPhone for like 3 years and I ve n   '

In [28]:
def denoise(text):
    return re.sub('[^a-zA-Z0-9 ]+',' ',text)

In [29]:
denoise("I've had an IPhone for like 3 years and I've n...")

'I ve had an IPhone for like 3 years and I ve n '

In [30]:
df['clean_text'] = df.clean_text.apply(denoise)
df.head()

,Text,Sentiment,clean_text
0,I've had an IPhone for like 3 years and I've n...,negative,iphone like 3 years never done hate ma life
1,I dont need the new iphone but I want it :(,negative,dont need new iphone want
2,fuck the iPhone 6s cus I'm not getting one :(,negative,fuck iphone 6s cus i m getting one
3,Hopefully will be able to get my iPhone 6s tod...,negative,hopefully able get iphone 6s today plane ge...
4,@clydesdalebank I've just bought a new iPhone ...,negative,clydesdalebank bought new iphone can t use ap...


In [31]:
x = vectorizer.fit_transform(df['clean_text'])
x.shape

(1002, 3135)

In [32]:
df['clean_text'] = df.Text.apply(denoise)
df['clean_text'] = df.clean_text.apply(sw_removal)
x = vectorizer.fit_transform(df['clean_text'])
x.shape

(1002, 3089)

In [33]:
train(x,df.Sentiment)

87.56

## Stemming

In [34]:
## play  playing  played  plays

In [36]:
!pip3 install nltk
from nltk.stem import PorterStemmer

     |████████████████████████████████| 1.5 MB 361 kB/s eta 0:00:01
     |████████████████████████████████| 764 kB 285 kB/s eta 0:00:01
     |████████████████████████████████| 76 kB 142 kB/s eta 0:00:01
  Attempting uninstall: regex
    Found existing installation: regex 2021.4.4
    Uninstalling regex-2021.4.4:
      Successfully uninstalled regex-2021.4.4


In [41]:
ps = PorterStemmer()
ps.stem('playing')

'play'

In [42]:
def stemm(text):
    clean_words = []
    for word in text.lower().split():
        clean_words.append(ps.stem(word))
        
    return ' '.join(clean_words)

In [44]:
stemm("fuck the iPhone 6s cus I'm not getting one :(	")

"fuck the iphon 6s cu i'm not get one :("

In [45]:
df['clean_text'] = df.Text.apply(denoise)
df['clean_text'] = df.clean_text.apply(sw_removal)
df['clean_text'] = df.clean_text.apply(stemm)

x = vectorizer.fit_transform(df['clean_text'])
x.shape

(1002, 2876)

In [50]:
train(x,df.Sentiment)

89.05

In [51]:
train(x[:,1556],df.Sentiment)

46.77

# Conclusion

In [52]:
df = pd.read_csv('iphone6-negative.csv')
df = df[['Text','Sentiment']]


sw = open('sw.txt','r').read().split('\n') ## \n --> refer to new line
def clean(text): ## text refer to whole sentence
    text = re.sub('[^a-zA-Z0-9 ]+',' ',text) #denoise
    wanted_words = []
    for word in text.lower().split():
        if word not in sw: #remove sw
            ## wanted word
            wanted_words.append(ps.stem(word)) #stemming
    return ' '.join(wanted_words) ## combine list to form sentence

def train(x,y):
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=110)
    clf = LogisticRegression()
    clf.fit(x_train,y_train)
    return round(clf.score(x_test,y_test)*100,2)


df['clean_text'] = df.Text.apply(clean)
x = vectorizer.fit_transform(df['clean_text'])
x.shape

train(x,df.Sentiment)

89.05